<a href="https://colab.research.google.com/github/goranagojic/paralelno-racunarstvo/blob/main/PR_termin6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Paralelno računarstvo - Vežba 6

Vežba obuhvata zadatke:

*   Zadatak 8 - Računanje vrednosti broja $\pi$
*   Zadaci 9 i 10 - Sekvencijalna i paralelna implementacija linearne pretrage
*   Zadaci 11 i 12 - Sekvencijalna i paralelna implementacija računanja histograma
*   Zadaci 13 i 14 - Sekvencijalna i paralelna implementacija množenja nekvadratne matrice i vektora
*   Zadaci 15 i 16 - Sekvencijalna i paralelna implementacija množenja dve kvadratne matrice



## Preduslovi

Pre nego započnete implementaciju zadatka, **obavezno** izvršite kod iz ćelije ispod. Kod će:

*   proveriti da li je na sistemu instalirana biblioteka `libgomp` potrebna za izvršavanje OpenMP programa i ako nije instaliraće je,
*   napraviti direktorijum u koji će se čuvati sve izvrne i izvršne datoteke
*   preuzeti datoteku `hpc_helpers.hpp` i sačuvati je na putanju /home/openmp/

In [ ]:
%%bash

# instaliraj libgomp biblioteku ukoliko vec nije instalirana
dpkg -l | grep libgomp1
[[ $? -ne 0 ]] && sudo apt-get install libgomp1 || echo "Biblioteka libgomp1 je već instalirana."

# napravi radni direktorijum
mkdir -p /home/openmp

# preuzmi hpc_helper zaglavlje
[[ -f hpc_helpers.zip ]] || wget -O hpc_helpers.zip "https://drive.google.com/u/0/uc?id=1PeEfh8h5SjEcQ5154Z6TpHIlJogQODwM&export=download"
unzip -oqqd /home/openmp/ hpc_helpers.zip


Po završetku prethodne ćelije možete pokrentui narednu koja proverava da li je hpc_helpers zaglavlje raspakovano na željenu putanju. Ukoliko je sve u redu, naredba će ispisati absolutnu putanju do zaglavlja.

In [ ]:
!ls /home/openmp/hpc_helpers.hpp

## Zadatak 8
Korišćenjem OpenMP direktiva, klauzula i funkcionalnosti OpenMP biblioteke implementirati paralelni C++ program za računanje vrednosti broja $\pi$ integraljenjem izraza:
$$
\int_{0}^{1}\frac{4}{(1+x^{2})}
$$
Rešenje implementirati u funkciji `pi_openmp_parallel` postavke date u ćeliji ispod.

Objašnjenje zadatka možete i poslušati na [ovom linku](https://youtu.be/FQ1k_YpyG_A?list=PLLX-Q6B8xqZ8n8bwjGdzBJ25X2utwnoEG&t=466).

In [ ]:
%%writefile /home/openmp/pi.cpp

#include <iostream>
#include <omp.h>

#include "hpc_helpers.hpp"

#define NOTIMPLEMENTED -1

static long num_steps = 100000000;


/*
  Referentno sekvencijalno resenje za racunanje vrednosti broja PI integracijom.
*/
double pi_sekvencijalno() {
    int i;
    double x, pi, sum = 0.0;

    double step = 1.0 / (double) num_steps;

    for (i = 0; i < num_steps; i++) { 
        double x = (i + 0.5) * step;
        sum = sum + 4.0 / (1.0 + x * x);    
    }
    pi = step * sum;

    return pi;
}


double pi_openmp_parallel() {

    // TODO ovde implementirati resenje zadatka 8

    return NOTIMPLEMENTED;
}


int main() {

    TIMERSTART(seq)
    double pi1 = pi_sekvencijalno();
    TIMERSTOP(seq)

    TIMERSTART(openmp_parallel)
    double pi2 = pi_openmp_parallel();
    TIMERSTOP(openmp_parallel)

    std::cout << "[pi_sekvencijalno]    pi = " << pi1 << std::endl;
    std::cout << "[pi_openmp_parallel2] pi = " << pi2 << std::endl;

    return 0;
}

### Kompajliranje izvornog koda

Pokrenite komandu u ćeliji ispod kako biste kompajlirali rešenje.

In [ ]:
%%bash

cd /home/openmp
g++ hpc_helpers.hpp pi.cpp -o pi -fopenmp

### [Opciono] Provera da li postoji izvršna datoteka

In [ ]:
![[ -f /home/openmp/pi ]] && echo "Postoji." || echo "Ne postoji."

### Pokretanje rešenja

Očekivani izlaz izvršne datoteke dobijene kompajliranjem postavke zadatka će ispisati:
*   vremena izvršavanja sekvencijalnog i paralelnog rešenja (vrlo mali broj kod paralelnog rešenja dok ne napišete samo rešenje) i
*   sračunatu vrednost broja $\pi$ - u slučaju paralelnog rešenja inicijalno se ispisuje -1 dok ne implementirate rešenje

In [ ]:
%%bash

cd /home/openmp
OMP_NUM_THREADS=16 ./pi

### [Opciono] Pokretanje rešenja sa različitim brojem niti

Nakon što se uverite da oba implementirano rešenje daje korektne rezultate, možete nastaviti sa izvršavanjem ćelija u ovom poglavlju. Svrha ovog poglavlja je da ilustruje zavisnost vremena izvršavanja od broja pokrenutih niti.

Prva ćelija u petlji poziva postojeću `pi` izvršnu datoteku, prvo zahtevajući dve niti, a pri svakom narednom pokretanju uvećavati broj niti dva puta sve dok ne stigne do vrednosti zadate promenljivom `MAX_THREADS`. Vrednost ove promenljive okruženja je postavljena na 256 i možete je menjati. Kako bi podaci bili ispravno sakupljeni, **neophodno** je da:
- sve ispise date u postavci zadatka zadržite takvim kakvi jesu i
- da dodate ispis ukupnog broja niti u paralelnom regionu. Ispis treba da vrši **samo jedna nit** i treba da bude u formatu `#threads: <n>` gde je `<n>` broj niti u paralelnom regionu (npr. `#threads: 16`).

In [ ]:
%%bash
 
cd /home/openmp
 
[[ -f /tmp/performance.out ]] && rm /tmp/performance.out
 
export MAX_THREADS=256 # vrednost mozete menjati
 
# prikupljanje podataka o brojnu niti i vremenu sekvencijalnog i paralelnog izvrsavanja
# podaci se cuvaju u /tmp/performance.out datoteci u formatu:
#   <sekvencijalno vreme (s)>, <broj niti>, <paralelno vreme (s)>
for (( i=2; i<$MAX_THREADS; i*=2 )) do
  export OMP_NUM_THREADS=$i;
  echo "Zahtevam ${OMP_NUM_THREADS} niti za pokretanje resenja..."
  out=$(./pi | sed -n 's/.*://p' | tr "\n" ",")
  echo ${out:0:-1} >> /tmp/performance.out
done
echo "Done."

Na osnovu skupljenih podataka se iscrtava grafik zavisnosti vremena izvršavanja u odnosu na broj niti (grafik skalabilnosti rešenja). Na `x` osi je broj niti u paralelnom regionu, a na `y` sekvencijalno (plavo) i paralelno (narandžasto) vreme izvršavanja dato u milisekundama.

In [ ]:
import pandas as pd
import numpy as np
 
# uklanja karakter sa kraja stringa, konvertuje string u razlomljeni broj i
# i mnozi ga sa 1000
# može se koristiti za konvertovanje stringovnog zapisa trajanja u sekundama u 
# milisekunde
get_exec_time = lambda x: float(x[:-1]) * 1000
 
# konvertuje string u celi broj
get_nthreads = lambda x: int(x)
 
# iscitava datoteku sa podacima izvrsavanja i iscrtava grafik
perf = pd.read_csv(
    filepath_or_buffer='/tmp/performance.out',
    delimiter=',',
    header=None,
    names=['seq', 'nthreads', 'parallel'],
    index_col=1,
    converters={
        'seq': get_exec_time,
        'parallel': get_exec_time,
        'nthreads': get_nthreads
    }
)
ax = perf.plot.bar(
    color={"parallel": "orange", "seq": "blue"},
    ylabel="time (ms)"
)
print(perf)

## Zadatak 9 - Linearna pretraga (sekvencijalna implementacija)

Implementirati sekvencijalni C++ program za linearnu pretragu vektora. Rešenje implementirati u telu funkcije `linearna_pretraga_sekvencijalno` u ćeliji ispod. Kao ulazne parametre, funkcija prima vektor vrednost i vrednost koje se traži u ulaznom vektoru. Vratiti indeks prve pronađene vrednosti u vektoru, odnosno -1 ukoliko tražena vrednost nije pronađena. Meriti vreme izvršavanja.

## Zadatak 10 - Linearna pretraga (paralelna implementacija)

Implementirati paralelno OpenMP C++ rešenje za linearnu pretragu ulaznog niza. Rešenje implementirati u telu funkcije `linearna_pretraga_openmp` ćelije sa postavkom zadatka. Meriti vreme izvršavanja. Uporediti vremena izvršavanja sekvencijalnog i paralelnog rešenja.

In [ ]:
%%writefile /home/openmp/linearna_pretraga.cpp

#include <algorithm>
#include <iostream>
#include <vector>
#include <climits>

#include "hpc_helpers.hpp"

#define NELEM 10000000
#define NOTIMPLEMENTED -1

using namespace std;


int linearna_pretraga_sekvencijalno(vector<int> &brojevi, int trazeni_broj) {

    // TODO Implementirati resenje zadatka 9

    return NOTIMPLEMENTED;
}


int linearna_pretraga_openmp(vector<int> &brojevi, int trazeni_broj) {

    // TODO Implementirati resenje zadatka 10

    return NOTIMPLEMENTED;
}


int main() {

    int trazeni_broj = 1;
    vector<int> brojevi(NELEM);

    // inicijalizacija vektora
    srand(time(NULL));

    // 1. inicijalizacija vektora nasumicnim brojevima razlicitim od trazenog broja zbog pouzdanijeg testiranja
    generate(brojevi.begin(), brojevi.end(), [=](){ return trazeni_broj + 1 + rand() % 100; });

    // 2. postavljanje n trazenih brojeva na nasumicne pozicije uz ispis indeksa vektora na koji je postavljen
    // broj koji ce se kasnije traziti. Istestirati resenje tako da je n = 0, n = 1 i n > 1.
    int n = 3;
    for (int i = 0; i < n; i++) {
        int poz = rand() % brojevi.size();
        brojevi[poz] = trazeni_broj;
        cout << "[init] Broj " << trazeni_broj << " postavljen na poziciju " << poz << " u nizu." << endl;
    }
    // KRAJ inicijalizacija vektora

    TIMERSTART(seq)
    int indeks1 = linearna_pretraga_sekvencijalno(brojevi, trazeni_broj);
    TIMERSTOP(seq)

    TIMERSTART(openmp)
    int indeks2 = linearna_pretraga_openmp(brojevi, trazeni_broj);
    TIMERSTOP(openmp)

    cout << "[sekvencijalno] Indeks prvog pojavljivanja broja: " << indeks1 << endl;
    cout << "[openmp] Indeks prvog pojavljivanja broja: " << indeks2 << endl;

    return 0;
}

### Kompajliranje izvornog koda

Pokrenite komandu u ćeliji ispod kako biste kompajlirali rešenje.

In [ ]:
%%bash

cd /home/openmp
g++ hpc_helpers.hpp linearna_pretraga.cpp -o linearna_pretraga -fopenmp

### [Opciono] Provera da li postoji izvršna datoteka

In [ ]:
![[ -f /home/openmp/linearna_pretraga ]] && echo "Postoji." || echo "Ne postoji."

### Pokretanje rešenja

Očekivani izlaz izvršne datoteke dobijene kompajliranjem postavke zadatka će ispisati:
*   indekse niza na kojima je postavljen broj koji se traži,
*   vremena izvršavanja sekvencijalnog i paralelnog rešenja (vrlo male vrednosti dok su obe funkcije neimplementirane) i
*   indeks prve pojave traženog broja za oba rešenja (podrazumevano -1 kada ni jedna od traženih funkcija nije implementirana)

In [ ]:
%%bash

cd /home/openmp
OMP_NUM_THREADS=2 ./linearna_pretraga

### [Opciono] Pokretanje rešenja sa različitim brojem niti

Nakon što se uverite da oba implementirano rešenje daje korektne rezultate, možete nastaviti sa izvršavanjem ćelija u ovom poglavlju. Svrha ovog poglavlja je da ilustruje zavisnost vremena izvršavanja od broja pokrenutih niti.

Prva ćelija u petlji poziva postojeću `linearna_pretraga` izvršnu datoteku, prvo zahtevajući dve niti, a pri svakom narednom pokretanju uvećavati broj niti dva puta sve dok ne stigne do vrednosti zadate promenljivom `MAX_THREADS`. Vrednost ove promenljive okruženja je postavljena na 256 i možete je menjati. Kako bi podaci bili ispravno sakupljeni, **neophodno** je da:
- sve ispise date u postavci zadatka zadržite takvim kakvi jesu i
- da dodate ispis ukupnog broja niti u paralelnom regionu. Ispis treba da vrši **samo jedna nit** i treba da bude u formatu `#threads: <n>` gde je `<n>` broj niti u paralelnom regionu (npr. `#threads: 16`).

In [ ]:
%%bash
 
cd /home/openmp
 
[[ -f /tmp/performance.out ]] && rm /tmp/performance.out
 
MAX_THREADS=256 # vrednost mozete menjati
 
# prikupljanje podataka o brojnu niti i vremenu sekvencijalnog i paralelnog izvrsavanja
# podaci se cuvaju u /tmp/performance.out datoteci u formatu:
#   <sekvencijalno vreme (s)>, <broj niti>, <paralelno vreme (s)>
for (( i=2; i<$MAX_THREADS; i*=2 )) do
  export OMP_NUM_THREADS=$i;
  echo "Zahtevam ${OMP_NUM_THREADS} niti za pokretanje resenja..."
  out=$(./pi | sed -n 's/.*://p' | tr "\n" ",")
  echo ${out:0:-1} >> /tmp/performance.out
done
echo "Done."

Na osnovu skupljenih podataka se iscrtava grafik zavisnosti vremena izvršavanja u odnosu na broj niti (grafik skalabilnosti rešenja). Na `x` osi je broj niti u paralelnom regionu, a na `y` sekvencijalno (plavo) i paralelno (narandžasto) vreme izvršavanja dato u milisekundama.

In [ ]:
import pandas as pd
import numpy as np
 
# uklanja karakter sa kraja stringa, konvertuje string u razlomljeni broj i
# i mnozi ga sa 1000
# može se koristiti za konvertovanje stringovnog zapisa trajanja u sekundama u 
# milisekunde
get_exec_time = lambda x: float(x[:-1]) * 1000
 
# konvertuje string u celi broj
get_nthreads = lambda x: int(x)
 
# iscitava datoteku sa podacima izvrsavanja i iscrtava grafik
perf = pd.read_csv(
    filepath_or_buffer='/tmp/performance.out',
    delimiter=',',
    header=None,
    names=['seq', 'nthreads', 'parallel'],
    index_col=1,
    converters={
        'seq': get_exec_time,
        'parallel': get_exec_time,
        'nthreads': get_nthreads
    }
)
ax = perf.plot.bar(
    color={"parallel": "orange", "seq": "blue"},
    ylabel="time (ms)"
)
print(perf)

## Zadatak 11 - Histogram (sekvencijalna implementacija)

Implementirati sekvencijalni C++ program za računanje histograma vrednosti iz ulaznog vektora. Vrednost u ulaznom vektoru su nasumično generisani brojevi immeđu 0 i 10. Postavka zadatka je data u želiji ispod gde je potrebno implementirati funkciju `histogram_sekvencijalno`. Meriti vreme izvršavanja.

## Zadatak 12 - Histogram (paralelna implementacija)

Implementirati paralelni OpenMP C++ program za računanje histograma za vektor ulaznih vrednosti. Rešenje implementirati kao telo funkcije `histogram_openmp`. Meriti vremena izvršavanja i rezultate paralelnog i sekvencijalnog rešenja i uporediti ih.

In [ ]:
%%writefile /home/openmp/histogram.cpp

#include <iostream>
#include <vector>
#include <algorithm>

#include "hpc_helpers.hpp"

#define NELEM 10000
#define NOTIMPLEMENTED vector<int>()


using namespace std;


vector<int> histogram_sekvencijalni(vector<int> &vrednosti) {
    
    // TODO implementirati resenje zadatka 11

    return NOTIMPLEMENTED;
}


vector<int> histogram_openmp(vector<int> &vrednosti) {

    // TODO implementirati resenje zadatka 12

    return NOTIMPLEMENTED;
}


int main() {

    vector<int> vrednosti(NELEM);

    // inicijalizacija vektora vrednosti nasumicnim vrednostima manjim od 10
    srand(time(NULL));
    generate(vrednosti.begin(), vrednosti.end(), [](){ return rand() % 10; });
    // KRAJ inicijalizacija

    TIMERSTART(seq)
    vector<int> histogram1 = histogram_sekvencijalni(vrednosti);
    TIMERSTOP(seq)

    TIMERSTART(openmp1)
    vector<int> histogram2 = histogram_openmp(vrednosti);
    TIMERSTOP(openmp1)

    if (!equal(histogram1.begin(), histogram1.end(), histogram2.begin())) {
        cout << "Rezultati sekvencijalnog i paralelnog izvrsavanja su razliciti!" << endl;
    }

    return 0;
}

### Kompajliranje izvornog koda

Pokrenite komandu u ćeliji ispod kako biste kompajlirali rešenje.

In [ ]:
%%bash
 
cd /home/openmp
g++ hpc_helpers.hpp histogram.cpp -o histogram -fopenmp

### [Opciono] Provera da li postoji izvršna datoteka

In [ ]:
![[ -f /home/openmp/histogram ]] && echo "Postoji." || echo "Ne postoji."

### Pokretanje rešenja

Očekivani izlaz izvršne datoteke dobijene kompajliranjem postavke zadatka će ispisati:
*   vremena izvršavanja sekvencijalnog i paralelnog rešenja (vrlo male vrednosti dok su obe funkcije neimplementirane)

In [ ]:
%%bash
 
cd /home/openmp
export OMP_NUM_THREADS=16; ./histogram

### [Opciono] Pokretanje rešenja sa različitim brojem niti

Nakon što se uverite da oba implementirana rešenja za računanje histograma daju korektne rezultate, možete nastaviti sa izvršavanjem ćelija u ovom poglavlju. Svrha ovog poglavlja je da ilustruje zavisnost vremena izvršavanja od broja pokrenutih niti.

Prva ćelija u petlji poziva postojeću `histogram` izvršnu datoteku, prvo zahtevajući dve niti, a pri svakom narednom pokretanju uvećavati broj niti dva puta sve dok ne stigne do vrednosti zadate promenljivom `MAX_THREADS`. Vrednost ove promenljive okruženja je postavljena na 256 i možete je menjati. Kako bi podaci bili ispravno sakupljeni, **neophodno** je da:
- sve ispise date u postavci zadatka zadržite takvim kakvi jesu i
- da dodate ispis ukupnog broja niti u paralelnom regionu. Ispis treba da vrši **samo jedna nit** i treba da bude u formatu `#threads: <n>` gde je `<n>` broj niti u paralelnom regionu (npr. `#threads: 16`).

In [ ]:
%%bash
 
cd /home/openmp
 
[[ -f /tmp/performance.out ]] && rm /tmp/performance.out
 
MAX_THREADS=256 # vrednost mozete menjati
 
# prikupljanje podataka o brojnu niti i vremenu sekvencijalnog i paralelnog izvrsavanja
# podaci se cuvaju u /tmp/performance.out datoteci u formatu:
#   <sekvencijalno vreme (s)>, <broj niti>, <paralelno vreme (s)>
for (( i=2; i<$MAX_THREADS; i*=2 )) do
  echo "Zahtevam $i niti za pokretanje resenja..."
  export OMP_NUM_THREADS=$i;
  out=$(./histogram | sed -n 's/.*://p' | tr "\n" ",")
  echo ${out:0:-1} >> /tmp/performance.out
done
echo "Done."

Na osnovu skupljenih podataka se iscrtava grafik zavisnosti vremena izvršavanja u odnosu na broj niti (grafik skalabilnosti rešenja). Na `x` osi je broj niti u paralelnom regionu, a na `y` sekvencijalno (plavo) i paralelno (narandžasto) vreme izvršavanja dato u milisekundama.

In [ ]:
import pandas as pd
import numpy as np
 
# uklanja karakter sa kraja stringa, konvertuje string u razlomljeni broj i
# i mnozi ga sa 1000
# može se koristiti za konvertovanje stringovnog zapisa trajanja u sekundama u 
# milisekunde
get_exec_time = lambda x: float(x[:-1]) * 1000
 
# konvertuje string u celi broj
get_nthreads = lambda x: int(x)
 
# iscitava datoteku sa podacima izvrsavanja i iscrtava grafik
perf = pd.read_csv(
    filepath_or_buffer='/tmp/performance.out',
    delimiter=',',
    header=None,
    names=['seq', 'nthreads', 'parallel'],
    index_col=1,
    converters={
        'seq': get_exec_time,
        'parallel': get_exec_time,
        'nthreads': get_nthreads
    }
)
ax = perf.plot.bar(
    color={"parallel": "orange", "seq": "blue"},
    ylabel="time (ms)"
)
print(perf)

## Zadatak 13 - Množenje matrice i vektora (sekvencijalna implementacija)

Implementirati C++ program za sekvencijalno množenje nekvadratne matrice i vektora. Rešenje implementirati u okviru metode `mnozenje_sekvencijalno` ćelije sa postavkom zadatka.

## Zadatak 14 - Množenje matrice i vektora (paralelna implementacija)

Implementirati OpenMP C++ program za paralelno množenje nekvadratne matrice i vektora. Rešenje implementirati u okviru metode `mnozenje_openmp` ćelije sa postavkom zadatka. Pre pokretanja rešenja otkomentarisati poziv funkcije iz glavnog koda i zakomentarisati poziv sekvencijalnog rešenja.

In [ ]:
%%writefile /home/openmp/matrix_vector_multiplication.cpp

#include <iostream>
#include <vector>

#include <omp.h>

#include "hpc_helpers.hpp"


template<typename value_t, typename index_t>
void init(
    std::vector<value_t> &matrica,
    std::vector<value_t> &vektor,
    index_t vrsta,                  // broj vrsta matrice
    index_t kolona                  // broj kolona matrice
) {
    for (index_t i = 0; i < vrsta; i++) {
        for (index_t j = 0; j < kolona; j++) {
            matrica[i * kolona + j] = i >= j ? 1 : 0;
        }
    }
    for (index_t j = 0; j < kolona; j++) {
        vektor[j] = j;
    }
}


template<typename value_t, typename index_t>
void mnozenje_sekvencijalno(
    std::vector<value_t> &matrica, 
    std::vector<value_t> &vektor, 
    std::vector<value_t> &rez, 
    index_t vrsta, 
    index_t kolona
) {
    // TODO implementirati rešenje zadatka 13
}


template<typename value_t, typename index_t>
void mnozenje_openmp(
    std::vector<value_t> &matrica, 
    std::vector<value_t> &vektor,
    std::vector<value_t> &rez, 
    index_t vrsta, 
    index_t kolona,
    const index_t broj_niti
) {
    
    // TODO implementirati rešenje zadatka 14
}


int main() {

    const unsigned int m = 1UL << 14;
    const unsigned int n = 1UL << 14;
    const unsigned int nt = 16UL;

    std::vector<int> matrica(m*n);
    std::vector<int> vektor(n);
    std::vector<int> rezultat(m);

    init(matrica, vektor, m, n);

    // zakomentarisati nakon sto implementirante mnozenje_openmp funkciju
    TIMERSTART(seq)
    mnozenje_sekvencijalno(matrica, vektor, rezultat, m, n);
    TIMERSTOP(seq)

    // otkomentarisati nakon sto implementirate mnozenje_openmp funkciju
    // TIMERSTART(openmp)
    // mnozenje_openmp(matrica, vektor, rezultat, m, n, nt);
    // TIMERSTOP(openmp)

    // ukoliko je rezultat u vektoru 'rezultat' tacan, ova petlja se
    // nece nista ispisati
    for (int i = 0; i < m; i++)
        if (rezultat[i] != i*(i+1)/2)
            std::cout << "greska na poziciji " << i << " "
                        << rezultat[i] << std::endl;


    return 0;
}

### Kompajliranje izvornog koda

Pokrenite komandu u ćeliji ispod kako biste kompajlirali rešenje.

In [ ]:
%%bash

cd /home/openmp
g++ hpc_helpers.hpp matrix_vector_multiplication.cpp -o matrix_vector_multiplication -fopenmp

### [Opciono] Provera da li postoji izvršna datoteka

In [ ]:
![[ -f /home/openmp/matrix_vector_multiplication ]] && echo "Postoji." || echo "Ne postoji."

### Pokretanje rešenja

Očekivani izlaz izvršne datoteke dobijene kompajliranjem postavke zadatka će ispisati:
*   vremene izvršavanja programa i 
*   dosta poruka o greškama na pozicijama u rezultujućoj matrici - ovo je u redu i ispisi će nestati nakon što pravilno implementirate algoritam množenja

In [ ]:
%%bash

cd /home/openmp
./matrix_vector_multiplication

### [Opciono] Pokretanje rešenja sa različitim brojem niti

Nakon što se uverite da oba implementirana rešenja za računanje proizvoda daju korektne rezultate, možete nastaviti sa izvršavanjem ćelija u ovom poglavlju. Svrha ovog poglavlja je da ilustruje zavisnost vremena izvršavanja od broja pokrenutih niti.

Prva ćelija u petlji poziva postojeću `matrix_vector_multiplication` izvršnu datoteku, prvo zahtevajući dve niti, a pri svakom narednom pokretanju uvećavati broj niti dva puta sve dok ne stigne do vrednosti zadate promenljivom `MAX_THREADS`. Vrednost ove promenljive okruženja je postavljena na 256 i možete je menjati. Kako bi podaci bili ispravno sakupljeni, **neophodno** je da:
- sve ispise date u postavci zadatka zadržite takvim kakvi jesu i
- da dodate ispis ukupnog broja niti u paralelnom regionu. Ispis treba da vrši **samo jedna nit** i treba da bude u formatu `#threads: <n>` gde je `<n>` broj niti u paralelnom regionu (npr. `#threads: 16`).

In [ ]:
%%bash

# prikupljanje podataka o brojnu niti i vremenu sekvencijalnog i paralelnog izvrsavanja
# podaci se cuvaju u /tmp/performance.out datoteci u formatu:
#   <sekvencijalno vreme (s)>, <broj niti>, <paralelno vreme (s)>
 
cd /home/openmp
 
[[ -f /tmp/performance.out ]] && rm /tmp/performance.out
 
MAX_THREADS=256 # vrednost mozete menjati
SRC=/home/openmp/
 
for (( i=2; i<$MAX_THREADS; i*=2 )) do
  echo "Zahtevam $i niti za pokretanje resenja..."

  # menja #define NELEM uz izvornoj datoteci na vrednost $i
  sed -i "s/const unsigned int nt =.*/const unsigned int nt = ${i};/" \
    ${SRC}/matrix_vector_multiplication.cpp >/dev/null;
  g++ -fopenmp $SRC/matrix_vector_multiplication.cpp -o matrix_vector_multiplication;
  out=$(./matrix_vector_multiplication | sed -n 's/.*://p' | tr "\n" ",")
  echo ${out:0:-1} >> /tmp/performance.out
done
echo "Done."

Na osnovu skupljenih podataka se iscrtava grafik zavisnosti vremena izvršavanja u odnosu na broj niti (grafik skalabilnosti rešenja). Na `x` osi je broj niti u paralelnom regionu, a na `y` sekvencijalno (plavo) i paralelno (narandžasto) vreme izvršavanja dato u milisekundama.

In [ ]:
import pandas as pd
import numpy as np
 
# uklanja karakter sa kraja stringa, konvertuje string u razlomljeni broj i
# i mnozi ga sa 1000
# može se koristiti za konvertovanje stringovnog zapisa trajanja u sekundama u 
# milisekunde
get_exec_time = lambda x: float(x[:-1]) * 1000
 
# konvertuje string u celi broj
get_nthreads = lambda x: int(x)
 
# iscitava datoteku sa podacima izvrsavanja i iscrtava grafik
perf = pd.read_csv(
    filepath_or_buffer='/tmp/performance.out',
    delimiter=',',
    header=None,
    names=['seq', 'nthreads', 'parallel'],
    index_col=1,
    converters={
        'seq': get_exec_time,
        'parallel': get_exec_time,
        'nthreads': get_nthreads
    }
)
ax = perf.plot.bar(
    color={"parallel": "orange", "seq": "blue"},
    ylabel="time (ms)"
)
print(perf)

## Zadatak 15 - Množenje matrica (sekvencijalna implementacija)

Implementirati C++ rešenje za sekvencijalno množenje dve kvadratne funkcije. Rešenje treba implementirati u okviru metode `mnozenje_sekvencijalno`.

## Zadatak 16 - Množenje matrica (paralelna implementacija)

Implementirati OpenMP C++ rešenje za paralelno množenje dve kavadratne matrice. Rešenje implementirati u okviru funkcije `mnozenje_openmp.` Pre pokretanja rešenja otkomentarisati poziv funkcije u funkciji `main` i zakomentarisati poziv sekvencijalnog rešenja.

In [ ]:
%%writefile /home/openmp/matrix_multiplication.cpp

#include <iostream>
#include <vector>

#include <omp.h>

#include "hpc_helpers.hpp"


template<typename value_t, typename index_t>
void init(
    std::vector<value_t> &A,
    std::vector<value_t> &B,
    index_t m,
    index_t n,
    index_t r
) {
    // inicijalizuje matricu A tako da su na glavnoj dijagonali i ispod
    // nje sve jedinice, a iznad glavne dijagonale sve nule
    for (index_t i = 0; i < m; i++) {
        for (index_t j = 0; j < n; j++) {
            A[i * m + j] = i >= j ? 1 : 0;
        }
    }
    
    // inicijalizuje matricu B tako da je svaka kolona popunjena rastucim
    // brojevima od 0 do n-1
    for (index_t j = 0; j < r; j++) {
        for (index_t i = 0; i < n; i++) {
            B[i * r + j] = i;
        }
    }
}


template<typename value_t, typename index_t>
void mnozenje_sekvencijalno(
    std::vector<value_t> &A, 
    std::vector<value_t> &B, 
    std::vector<value_t> &R, 
    index_t m, 
    index_t n,
    index_t r
) {
    // TODO Implementirati resenje zadatka 15
}


template<typename value_t, typename index_t>
void mnozenje_openmp(
    std::vector<value_t> &A, 
    std::vector<value_t> &B,
    std::vector<value_t> &R, 
    index_t m, 
    index_t n,
    index_t r,
    const index_t broj_niti
) {
    // TODO Implementirati resenje zadatka 16
}


int main() {

    const uint8_t shift = 2;

    const unsigned int m = 1UL << shift;
    const unsigned int n = 1UL << shift;
    const unsigned int r = 1UL << shift;
    const unsigned int nt = 16UL;

    TIMERSTART(overall)

    TIMERSTART(alloc)
    std::vector<int> A(m*n);
    std::vector<int> B(n*r);
    std::vector<int> R(m*r);
    TIMERSTOP(alloc)

    init(A, B, m, n, r);

    // zakomentarisati nakon sto implementirante mnozenje_openmp funkciju
    TIMERSTART(seq)
    mnozenje_sekvencijalno(A, B, R, m, n, r);
    TIMERSTOP(seq)

    // otkomentarisati nakon sto implementirate mnozenje_openmp funkciju
    // TIMERSTART(openmp)
    // mnozenje_openmp(A, B, R, m, n, r, nt);
    // TIMERSTOP(openmp)

    TIMERSTOP(overall)

    // ukoliko je rezultat u rezultujucoj matrici tacan, ova petlja nece nista ispisati
    for (int j = 0; j < r; j++)
        for (int i = 0; i < m; i++)
            if (R[i*m+j] != i*(i+1)/2)
                std::cout << "greska na poziciji " << i << " "
                            << R[i] << std::endl;


    return 0;
}

### Kompajliranje izvornog koda

Pokrenite komandu u ćeliji ispod kako biste kompajlirali rešenje.

In [ ]:
%%bash

cd /home/openmp
g++ hpc_helpers.hpp matrix_multiplication.cpp -o matrix_multiplication -fopenmp

### [Opciono] Provera da li postoji izvršna datoteka

In [ ]:
![[ -f /home/openmp/matrix_multiplication ]] && echo "Postoji." || echo "Ne postoji."

### Pokretanje rešenja

Očekivani izlaz izvršne datoteke dobijene kompajliranjem postavke zadatka će ispisati:
*   vremene izvršavanja programa i 
*   dosta poruka o greškama na pozicijama u rezultujućoj matrici - ovo je u redu i ispisi će nestati nakon što pravilno implementirate algoritam množenja. Nakon sto ispravno implementirate rešenje, probajte da povećate vrednost promenljive `shift` i pratite performanse rešenja.

In [ ]:
%%bash

cd /home/openmp
./matrix_multiplication

### [Opciono] Pokretanje rešenja sa različitim brojem niti

Nakon što se uverite da oba implementirana rešenja za računanje proizvoda daju korektne rezultate, možete nastaviti sa izvršavanjem ćelija u ovom poglavlju. Svrha ovog poglavlja je da ilustruje zavisnost vremena izvršavanja od broja pokrenutih niti.

Prva ćelija u petlji poziva postojeću `matrix_multiplication` izvršnu datoteku, prvo zahtevajući dve niti, a pri svakom narednom pokretanju uvećavati broj niti dva puta sve dok ne stigne do vrednosti zadate promenljivom `MAX_THREADS`. Vrednost ove promenljive okruženja je postavljena na 256 i možete je menjati. Kako bi podaci bili ispravno sakupljeni, **neophodno** je da:
- sve ispise date u postavci zadatka zadržite takvim kakvi jesu i
- da dodate ispis ukupnog broja niti u paralelnom regionu. Ispis treba da vrši **samo jedna nit** i treba da bude u formatu `#threads: <n>` gde je `<n>` broj niti u paralelnom regionu (npr. `#threads: 16`).

In [ ]:
%%bash

# prikupljanje podataka o brojnu niti i vremenu sekvencijalnog i paralelnog izvrsavanja
# podaci se cuvaju u /tmp/performance.out datoteci u formatu:
#   <sekvencijalno vreme (s)>, <broj niti>, <paralelno vreme (s)>
 
cd /home/openmp
 
[[ -f /tmp/performance.out ]] && rm /tmp/performance.out
 
MAX_THREADS=256 # vrednost mozete menjati
SRC=/home/openmp/
 
for (( i=2; i<$MAX_THREADS; i*=2 )) do
  echo "Zahtevam $i niti za pokretanje resenja..."

  # menja #define NELEM uz izvornoj datoteci na vrednost $i
  sed -i "s/const unsigned int nt =.*/const unsigned int nt = ${i};/" \
    ${SRC}/matrix_multiplication.cpp >/dev/null;
  g++ -fopenmp $SRC/matrix_multiplication.cpp -o matrix_multiplication;
  out=$(./matrix_multiplication | sed -n 's/.*://p' | tr "\n" ",")
  echo ${out:0:-1} >> /tmp/performance.out
done
echo "Done."

Na osnovu skupljenih podataka se iscrtava grafik zavisnosti vremena izvršavanja u odnosu na broj niti (grafik skalabilnosti rešenja). Na `x` osi je broj niti u paralelnom regionu, a na `y` sekvencijalno (plavo) i paralelno (narandžasto) vreme izvršavanja dato u milisekundama.

In [ ]:
import pandas as pd
import numpy as np
 
# uklanja karakter sa kraja stringa, konvertuje string u razlomljeni broj i
# i mnozi ga sa 1000
# može se koristiti za konvertovanje stringovnog zapisa trajanja u sekundama u 
# milisekunde
get_exec_time = lambda x: float(x[:-1]) * 1000
 
# konvertuje string u celi broj
get_nthreads = lambda x: int(x)
 
# iscitava datoteku sa podacima izvrsavanja i iscrtava grafik
perf = pd.read_csv(
    filepath_or_buffer='/tmp/performance.out',
    delimiter=',',
    header=None,
    names=['seq', 'nthreads', 'parallel'],
    index_col=1,
    converters={
        'seq': get_exec_time,
        'parallel': get_exec_time,
        'nthreads': get_nthreads
    }
)
ax = perf.plot.bar(
    color={"parallel": "orange", "seq": "blue"},
    ylabel="time (ms)"
)
print(perf)